In [1]:
# Exploring the Data
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re


In [2]:
df = pd.read_csv('/home/sarang/Downloads/btc_prediction_project/BTC_tweets_daily_example.csv')
print(df['Sentiment'])

0         ['neutral']
1         ['neutral']
2        ['positive']
3        ['positive']
4        ['positive']
5        ['positive']
6        ['positive']
7         ['neutral']
8        ['positive']
9        ['positive']
10       ['positive']
11        ['neutral']
12        ['neutral']
13        ['neutral']
14       ['positive']
15       ['positive']
16       ['positive']
17       ['positive']
18        ['neutral']
19        ['neutral']
20       ['positive']
21        ['neutral']
22        ['neutral']
23       ['positive']
24       ['positive']
25       ['positive']
26        ['neutral']
27       ['positive']
28        ['neutral']
29       ['negative']
             ...     
50829    ['positive']
50830     ['neutral']
50831     ['neutral']
50832    ['positive']
50833     ['neutral']
50834     ['neutral']
50835    ['positive']
50836     ['neutral']
50837    ['negative']
50838     ['neutral']
50839     ['neutral']
50840     ['neutral']
50841    ['positive']
50842     ['neutral']
50843     

In [3]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    if type(text) is str:
        text = text.lower() # lowercase text
    text = re.sub("[^a-zA-Z]"," ", str(text))
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['Tweet'] = df['Tweet'].apply(clean_text)
df['Tweet'].apply(lambda x: len(x.split(' '))).sum()
df['Sentiment'] = df['Sentiment'].apply(clean_text)
df['Sentiment'].apply(lambda x: len(x.split(' '))).sum()

50873

In [4]:
df['Sentiment']

0         neutral
1         neutral
2        positive
3        positive
4        positive
5        positive
6        positive
7         neutral
8        positive
9        positive
10       positive
11        neutral
12        neutral
13        neutral
14       positive
15       positive
16       positive
17       positive
18        neutral
19        neutral
20       positive
21        neutral
22        neutral
23       positive
24       positive
25       positive
26        neutral
27       positive
28        neutral
29       negative
           ...   
50829    positive
50830     neutral
50831     neutral
50832    positive
50833     neutral
50834     neutral
50835    positive
50836     neutral
50837    negative
50838     neutral
50839     neutral
50840     neutral
50841    positive
50842     neutral
50843     neutral
50844     neutral
50845     neutral
50846     neutral
50847    positive
50848     neutral
50849    negative
50850    positive
50851     neutral
50852    positive
50853     

In [5]:
df['Sentiment']

0         neutral
1         neutral
2        positive
3        positive
4        positive
5        positive
6        positive
7         neutral
8        positive
9        positive
10       positive
11        neutral
12        neutral
13        neutral
14       positive
15       positive
16       positive
17       positive
18        neutral
19        neutral
20       positive
21        neutral
22        neutral
23       positive
24       positive
25       positive
26        neutral
27       positive
28        neutral
29       negative
           ...   
50829    positive
50830     neutral
50831     neutral
50832    positive
50833     neutral
50834     neutral
50835    positive
50836     neutral
50837    negative
50838     neutral
50839     neutral
50840     neutral
50841    positive
50842     neutral
50843     neutral
50844     neutral
50845     neutral
50846     neutral
50847    positive
50848     neutral
50849    negative
50850    positive
50851     neutral
50852    positive
50853     

In [6]:
my_tags = ['positive','negative','neutral']
df['Sentiment'] = df['Sentiment'].factorize()[0]

In [10]:
#Conversion of text into Feature vectors

X = df.Tweet
y = df.Sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

39739    https co xugcgghvsv sohu ico solution agency a...
27280    rt airdropkitties register amp go long short e...
27439    vlc airdrop get free token vlc https co awzljs...
23463    rt bethereumteam successful launch bounty camp...
45508    dandoncrypto bitcoin btg bitcoin gold go gaini...
24965    could bitcoin gold benefit weakness stock mark...
31633    grockrecords bitcoin gold bitcoin gold become ...
45563    bitcoin price usd blx https co ubgkl pwso http...
46040    rt eiracube square seeks bitlicense bring bitc...
2191     rt bethereumteam smart contracts read technolo...
45049    bitcoin bitcoin cash ethereum bitcoin ethereum...
10092    moneyandpower bitcoin better money https co ug...
24799    check list trusted cloud mining services opini...
3494     rt randolphmlny bitcoin crypto blockchain aird...
25926    rt btccloud official bitcoin cloud airdrop htt...
45371    name dadi symbol dadi hour change price rank t...
22223              rt rnr bitcoin care https co oqvdiutm

In [ ]:
#Naive Bayes Classifier for Multinomial Models

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
print(X_train)
print(y_train)
nb.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('Naive Bayes accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=my_tags))
naivebayes_accuracy = accuracy_score(y_pred, y_test)

# save the model to disk
import pickle
filename = 'naive_bayes_model.sav'
pickle.dump(nb, open(filename, 'wb'))

In [ ]:
#Linear SVC
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)


y_pred = sgd.predict(X_test)

print('Linear SVC accuracy %s' % accuracy_score(y_pred, y_test))
linearsvc_accuracy = accuracy_score(y_pred, y_test)
#print(classification_report(y_test, y_pred,target_names=my_tags))
# save the model to disk
import pickle
filename = 'svm_model.sav'
pickle.dump(sgd, open(filename, 'wb'))

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

clf1 = nb
clf2 = sgd


logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])

clf3 = logreg.fit(X_train, y_train)

eclf1 = VotingClassifier(estimators=[('nb', clf1),('log', clf3)], voting='hard')
eclf1.fit(X_train, y_train)
y_pred = eclf1.predict(X_test)

print('Hybrid Classifier %s' % accuracy_score(y_pred, y_test))
log_nb_accuracy = accuracy_score(y_pred, y_test)

import pickle
filename = 'hybrid_model.sav'
pickle.dump(clf3, open(filename, 'wb'))

In [ ]:
#Comparison chart of different accuracies
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style

style.use('fivethirtyeight')

N = 1
ind = np.arange(N)  # the x locations for the groups
width = 0.5       # the width of the bars

fig, ax = plt.subplots(figsize=(8,7))

linearsvc_percentage = (linearsvc_accuracy) * 100
rects1 = ax.bar(ind, linearsvc_percentage, width, color='r')

niavebayes_percentage = naivebayes_accuracy * 100
rects2 = ax.bar(ind+width, niavebayes_percentage, width, color='y')

logisticregression_nb_percentage = log_nb_accuracy * 100
rects3 = ax.bar(ind+width*2, logisticregression_nb_percentage, width, color='k')




# add some text for labels, title and axes ticks
ax.set_xlabel('Classifier')
ax.set_ylabel('Accuracy (by percentage)')
ax.set_title('Accuracy by Diff Classifiers')
ax.set_xticks(ind+width)
ax.set_xticklabels( ('') )

ax.legend( (rects1[0], rects2[0],rects3[0]), ('Linear SVC', 'Naive Bayes', 'NB + LogReg'), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0. )

def autolabel(rects):
	# attach some text labels
	for rect in rects:
		height = rect.get_height()
		ax.text(rect.get_x()+rect.get_width()/2., 1.02*height, '%10.2f' % float(height),
				ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)

plt.show()
